In [1]:
import json
import pandas as pd 
from collections import defaultdict
from dateutil.relativedelta import relativedelta

In [8]:

# Path to your JSON file
# file_path = 'Credit_bureau_sample_data.json'
file_path = 'credit_report_sample.json'


# Function to load JSON data from a file
def load_json_data(file_path):
    with open(file_path, 'r') as file:
        json_data = json.load(file)
    return json_data

# Example usage
json_data = load_json_data(file_path)


In [9]:
import json
from datetime import datetime

class CreditReportFeatureExtractor:
    def __init__(self, json_data):
        self.data = json.loads(json_data)
        
    def extract_features(self):
        features = []
        for report in self.data:
            report_data = report['data']['consumerfullcredit']
            personal_details = report_data.get('personaldetailssummary', {})
            credit_summary = report_data.get('creditaccountsummary', {})
            credit_agreements = report_data.get('creditagreementsummary', [])
            employment_history = report_data.get('employmenthistory', [])
            features.append({
                'application_id': report['application_id'],
                'total_accounts_good': self._total_accounts_good(report_data),
                'total_accounts_bad': self._total_accounts_bad(report_data),
                'total_outstanding_debt': self._total_outstanding_debt(report_data),
                'total_enquiries': len(report_data.get('enquiryhistorytop', [])),
                'guarantor_count': int(report_data.get('guarantorcount', {}).get('accounts', 0)),
                'delinquency_months_in_arrears': self._delinquency_months_in_arrears(report_data),
                'employment_stability': self._employment_stability(report_data),
                'credit_utilization_ratio': self._credit_utilization_ratio(report_data),
                'recent_credit_activity': self._recent_credit_activity(report_data),
                'length_of_credit_history': self._length_of_credit_history(report_data),
                'gender': personal_details.get('gender', None),
                'age': self._calculate_age(personal_details.get('birthdate', '1900-01-01')),
                'total_accounts': int(credit_summary.get('totalaccounts', 0)),
                'total_account_arrear': int(credit_summary.get('totalaccountarrear', 0)),
                'total_monthly_instalment': float(credit_summary.get('totalmonthlyinstalment', '0').replace(',', '')),
                'nationality': personal_details.get('nationality', None),
                'number_of_dependants': int(personal_details.get('dependants', 0)),
                'total_credit_enquiries': self._total_credit_enquiries(report_data),
                'total_current_balance_amount': self._total_current_balance_amount(credit_agreements),
                'average_loan_duration': self._average_loan_duration(credit_agreements),
                'average_monthly_installment': self._average_monthly_installment(credit_agreements),
                'employer_detail': self._latest_employer_detail(employment_history),
                'occupation': self._latest_occupation(employment_history),
                'total_dishonoured_amounts': self._total_dishonoured_amounts(report_data),
                'employment_length': self._employment_length(report_data),
                'quarterly_credit_enquiries': self._quarterly_credit_enquiries(report_data),
                'quarterly_new_accounts': self._quarterly_new_accounts(report_data),
            })
        return features

    def _calculate_age(self, birthdate_str):
        birthdate = datetime.strptime(birthdate_str, '%d/%m/%Y')
        today = datetime.today()
        return today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    
    def _total_credit_enquiries(self, report_data):
        return len(report_data.get('enquiryhistorytop', []))
    
    def _employment_length(self, report_data):
        employment_histories = report_data.get('employmenthistory', [])
        if employment_histories:
            dates = [datetime.strptime(hist.get('updatedate', '1900-01-01'), '%d/%m/%Y') for hist in employment_histories if 'updatedate' in hist]
            if dates:
                earliest_date = min(dates)
                return (datetime.now() - earliest_date).days // 365
        return 0
    
    def _total_accounts_good(self, report_data):
        return sum(int(report_data['accountrating'][key]) for key in report_data['accountrating'] if 'good' in key)

    def _total_accounts_bad(self, report_data):
        return sum(int(report_data['accountrating'][key]) for key in report_data['accountrating'] if 'bad' in key)
    
    def _total_outstanding_debt(self, report_data):
        return float(report_data['creditaccountsummary']['totaloutstandingdebt'].replace(',', ''))
    
    def _delinquency_months_in_arrears(self, report_data):
        return int(report_data.get('deliquencyinformation', {}).get('monthsinarrears', 0))
    
    def _employment_stability(self, report_data):
        # Simplified example: count the number of distinct employers
        employers = set(hist.get('employerdetail', '') for hist in report_data.get('employmenthistory', []))
        return len(employers)
    
    def _credit_utilization_ratio(self, report_data):
        # Simplified calculation: current balance / opening balance for all accounts
        total_current_balance = sum(float(acc['currentbalanceamt'].replace(',', '')) for acc in report_data.get('creditagreementsummary', []))
        total_opening_balance = sum(float(acc['openingbalanceamt'].replace(',', '')) for acc in report_data.get('creditagreementsummary', []))
        return (total_current_balance / total_opening_balance) if total_opening_balance > 0 else 0

    def _recent_credit_activity(self, report_data):
        # Simplified: count of enquiries in the last 6 months
        recent_enquiries = [e for e in report_data.get('enquiryhistorytop', []) if (datetime.now() - datetime.strptime(e['daterequested'], '%d/%m/%Y %H:%M:%S')).days < 180]
        return len(recent_enquiries)
    
    def _length_of_credit_history(self, report_data):
        # Date of the earliest account opened
        dates = [datetime.strptime(acc['dateaccountopened'], '%d/%m/%Y') for acc in report_data.get('creditagreementsummary', [])]
        if dates:
            earliest_date = min(dates)
            return (datetime.now() - earliest_date).days / 365.25
        return 0
    

    def _total_current_balance_amount(self, credit_agreements):
        return sum(float(acc.get('currentbalanceamt', '0').replace(',', '')) for acc in credit_agreements)
    
    def _average_loan_duration(self, credit_agreements):
        durations = [int(acc.get('loanduration', '0')) for acc in credit_agreements if acc.get('loanduration', '0').isdigit()]
        return sum(durations) / len(durations) if durations else 0
    
    def _average_monthly_installment(self, credit_agreements):
        installments = [float(acc.get('instalmentamount', '0').replace(',', '')) for acc in credit_agreements]
        return sum(installments) / len(installments) if installments else 0
    
    def _latest_employer_detail(self, employment_history):
        if employment_history:
            return employment_history[0].get('employerdetail', 'N/A')
        return 'N/A'
    
    def _latest_occupation(self, employment_history):
        if employment_history:
            return employment_history[0].get('occupation', 'N/A')
        return 'N/A'
    
    def _total_dishonoured_amounts(self, report_data):
        summary = report_data.get('creditaccountsummary', {})
        return float(summary.get('totaldishonouredamount', '0').replace(',', ''))
    
    def _indicator_description(self, credit_agreements):
        descriptions = {acc.get('indicatordescription', 'N/A') for acc in credit_agreements}
        return ', '.join(descriptions)
        
    def _quarterly_credit_enquiries(self, report_data):
        enquiries = report_data.get('enquiryhistorytop', [])
        return self._items_per_last_four_quarters(enquiries, 'daterequested')
    
    def _quarterly_new_accounts(self, report_data):
        accounts = report_data.get('creditagreementsummary', [])
        return self._items_per_last_four_quarters(accounts, 'dateaccountopened')
    
    def _items_per_last_four_quarters(self, items, date_field):
        quarter_counts = [0, 0, 0, 0]
        for item in items:
            date_str = item.get(date_field, "")
            if date_str:
                try:
                    # Ensure your date format matches here
                    date = datetime.strptime(date_str.split(" ")[0], "%d/%m/%Y")
                    quarter, year = self._get_quarter_and_year(date)
                    current_quarter, current_year = self._get_quarter_and_year(datetime.now())
                    
                    quarter_diff = (current_year - year) * 4 + (current_quarter - quarter)
                    
                    if 0 <= quarter_diff < 4:
                        quarter_counts[quarter_diff] += 1
                        
                except ValueError as e:
                    print(f"Date parsing error: {e}")
        
        return quarter_counts
    
    def _get_quarter_and_year(self, date):
        quarter = (date.month - 1) // 3 + 1
        return quarter, date.year



The Class will work as a Json Extractor and will extract all the 27 Features mentioned below. We just need to Point the path of Jsons 

In [10]:
extractor = CreditReportFeatureExtractor(json.dumps(json_data))  # Convert the Python dict back to a JSON string
features = extractor.extract_features()

# Convert the list of dictionaries to a DataFrame
features_df = pd.DataFrame(features)

# Reordering the DataFrame to have application_id as the first column if it's not already
cols = ['application_id'] + [col for col in features_df.columns if col != 'application_id']
features_df = features_df[cols]


In [11]:
features_df

,application_id,total_accounts_good,total_accounts_bad,total_outstanding_debt,total_enquiries,guarantor_count,delinquency_months_in_arrears,employment_stability,credit_utilization_ratio,recent_credit_activity,...,total_credit_enquiries,total_current_balance_amount,average_loan_duration,average_monthly_installment,employer_detail,occupation,total_dishonoured_amounts,employment_length,quarterly_credit_enquiries,quarterly_new_accounts
0,9711360,7,0,105435.0,15,0,13,3,0.091813,0,...,15,146825.79,1014.714286,11057.782857,ALL MILITARY STAFFS,PUBLIC SERVANTS,0.0,8,"[0, 0, 0, 0]","[0, 0, 0, 0]"
1,9714953,17,0,294770.0,10,0,2,4,0.474212,0,...,10,294770.06,121.882353,7775.032353,None,CIVIL SERVANT,0.0,9,"[0, 0, 0, 0]","[0, 0, 0, 0]"
2,9714978,2,1,110919.0,10,0,109,1,0.101673,0,...,10,110919.00,125.000000,2333.333333,N/A,STUDENT,0.0,0,"[0, 0, 0, 0]","[0, 0, 0, 0]"


In [12]:
print(f"Total Number of Fetures Extracted:", len(list(features_df.columns))) 

Total Number of Fetures Extracted: 28


Certainly, defining each feature in simple terms can help understand their relevance to credit scoring models. Here are definitions for the 28 features we've discussed or incorporated:

1. **Total Accounts Good**: The total number of credit accounts in good standing (not late or in default).
2. **Total Accounts Bad**: The number of credit accounts that are late, in default, or have been written off.
3. **Total Outstanding Debt**: The sum of all amounts currently owed across all accounts.
4. **Total Enquiries**: The total number of times a credit report has been requested, typically due to a loan application.
5. **Guarantor Count**: The number of accounts for which the applicant is a guarantor.
6. **Delinquency Months in Arrears**: The number of months an account has been overdue.
7. **Employment Stability**: A measure of how stable an applicant's employment is, inferred from the variety of employment details provided.
8. **Credit Utilization Ratio**: The ratio of current credit balance to the credit limit, indicating how much of the available credit is being used.
9. **Recent Credit Activity**: The number of new credit inquiries or accounts opened in the recent past, indicating recent credit-seeking behavior.
10. **Length of Credit History**: The duration since the first credit account was opened, indicating experience with managing credit.
11. **Gender**: The applicant's gender, which may have statistical correlations with certain credit behaviors.
12. **Age**: The applicant's age, calculated from their birthdate.
13. **Total Accounts**: The total number of credit accounts, regardless of status.
14. **Total Account Arrear**: The total number of accounts that are past due.
15. **Total Monthly Instalment**: The total monthly payment due across all credit accounts.
16. **Nationality**: The applicant's nationality.
17. **Number of Dependants**: The number of people financially dependent on the applicant.
18. **Total Credit Enquiries**: The total number of times a credit report has been pulled, similar to Total Enquiries but may include a specific time frame or type of enquiry.
19. **Employment Length**: The duration the applicant has been with their current employer.
20. **Quarterly Credit Enquiries**: The number of credit enquiries made in each of the last four quarters, indicating recent seeking of new credit.
21. **Quarterly New Accounts**: The number of new credit accounts opened in each of the last four quarters, indicating recent credit account acquisitions.
22. **Total Current Balance Amount**: The total current balance across all credit accounts, reflecting the applicant's current total debt.
23. **Loan Duration**: The term length of the loan(s), indicating how long the applicant is committed to payments.
24. **Avg Monthly Installment**: The average monthly payment across all loans, giving an idea of the regular financial obligation due to credit.
25. **Employer Detail**: Information about the applicant's employer, potentially indicating job stability and income level.
26. **Occupation**: The applicant's job role, which can be an indicator of income stability and level.
27. **Total Dishonoured Amounts**: The total amount of payments that have failed due to insufficient funds or other reasons, indicating potential financial stress.


These features collectively provide a comprehensive picture of an applicant's financial situation, credit behavior, and potential risk as a borrower. They are used in credit scoring models to evaluate the likelihood that an applicant will repay borrowed money.
